In [1]:
!wget https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv

--2025-06-14 17:25:08--  https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv [following]
--2025-06-14 17:25:09--  https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2609524 (2.5M) [text/plain]
Saving to: ‘dirty_store_transactions.csv.2’

100%[======================================>] 2,609,524   2.56MB/s   in 1.0s   

2025-06-14 17:25:10 (2.56 MB/s) - ‘dirty_store_transactions

In [1]:
import findspark

In [2]:
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
 .appName("Spark App Data Cleaning") \
 .master("yarn") \
 .enableHiveSupport() \
.getOrCreate()

In [6]:
df = spark.read \
    .option("header",True) \
    .option("sep", ",") \
    .option("inferSchema", True) \
    .csv("file:///home/train/Calisma_Alanim/dirty_store_transactions.csv.2")

In [7]:
df.limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York(,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York+,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York!,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


## Görev1: 

dirty_store_transactions.csv verisetini kullanarak aşağıdaki işleri yapan bir spark uygulaması yazıp, spark-submit ile çalıştırınız.

### Adım1: Veriyi temizleyiniz ve yapısal hale getiriniz.

In [8]:
df1 = df.withColumn("STORE_LOCATION", F.regexp_replace(F.col("STORE_LOCATION"),"[^A-Za-z0-9]", "")) \
        .withColumn("PRODUCT_ID", F.regexp_replace(F.col("PRODUCT_ID"), "[^A-Za-z0-9]", ""))

df1.limit(3).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,NewYork,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,NewYork,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,NewYork,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26


In [9]:
df2 = df1.withColumn("MRP", F.regexp_replace(F.col("MRP"), "\$", "").cast(FloatType())) \
         .withColumn("CP", F.regexp_replace(F.col("CP"), "\$", "").cast(FloatType())) \
         .withColumn("SP", F.regexp_replace(F.col("SP"), "\$", "").cast(FloatType())) \
         .withColumn("DISCOUNT", F.regexp_replace(F.col("DISCOUNT"), "\$", "").cast(FloatType())) \
         .withColumn("Date_Casted", F.col("Date").cast(DateType())).drop("Date")

df2.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: float (nullable = true)
 |-- CP: float (nullable = true)
 |-- DISCOUNT: float (nullable = true)
 |-- SP: float (nullable = true)
 |-- Date_Casted: date (nullable = true)



In [10]:
df2.limit(3).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date_Casted
0,YR7220,NewYork,Electronics,12254943,31.0,20.77,1.86,29.139999,2019-11-26
1,YR7220,NewYork,Furniture,72619323C,15.0,9.75,1.50,13.500000,2019-11-26
2,YR7220,NewYork,Electronics,34161682B,88.0,62.48,4.40,83.599998,2019-11-26


### Adım2:Temiz veriyi hive'a test1.clean_transactions adıyla orc tablosu olarak yazınız.

In [11]:
spark.sql("show databases;").show()

+---------+
|namespace|
+---------+
|bookstore|
|  default|
+---------+



In [12]:
spark.sql("show tables;").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|         advertising|      false|
| default|      market1mil_tbl|      false|
| default|market5mil_pby_re...|      false|
| default|      market5mil_tbl|      false|
| default|     order_items_tbl|      false|
| default|          orders_tbl|      false|
+--------+--------------------+-----------+



In [13]:
spark.sql("create database if not exists test1")

DataFrame[]

In [14]:
spark.sql("use test1;")

DataFrame[]

In [15]:
# Hive'a yazdırma
import time

start_time = time.time()

df2.write \
.format("orc") \
.mode("overwrite") \
.saveAsTable("test1.clean_transactions")

print("%s secs = "% (time.time() - start_time))

4.209304332733154 secs = 


In [16]:
spark.sql(" select * from test1.clean_transactions").limit(10).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date_Casted
0,YR7220,NewYork,Electronics,12254943,31.0,20.770000,1.86,29.139999,2019-11-26
1,YR7220,NewYork,Furniture,72619323C,15.0,9.750000,1.50,13.500000,2019-11-26
2,YR7220,NewYork,Electronics,34161682B,88.0,62.480000,4.40,83.599998,2019-11-26
3,YR7220,NewYork,Kitchen,79411621,91.0,58.240002,3.64,87.360001,2019-11-26
4,YR7220,NewYork,Fashion,39520263T,85.0,51.000000,2.55,82.449997,2019-11-26
5,YR7220,NewYork,Kitchen,93809204,37.0,24.049999,0.74,36.259998,2019-11-26
6,YR7220,NewYork,Cosmetics,86610412D,80.0,48.799999,6.40,73.599998,2019-11-26
7,YR7220,NewYork,Kitchen,52503356,71.0,42.599998,5.68,65.320000,2019-11-26
8,YR7220,NewYork,Kitchen,77516479,92.0,56.119999,3.68,88.320000,2019-11-26
9,YR7220,NewYork,Cosmetics,47334289,16.0,10.720000,0.96,15.040000,2019-11-26


### Adım3:Temiz veriyi Postgresql'e traindb.public.clean_transactions tablosuna yazınız.

In [17]:
jdbcUrl = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [20]:
df2.write \
.jdbc(url=jdbcUrl, table="clean_transactions", mode="overwrite", properties={"driver":"org.postgresql.Driver"})

In [21]:
spark.read.format("jdbc") \
    .option("url", jdbcUrl) \
    .option("query", "SELECT * FROM clean_transactions") \
    .option("driver", "org.postgresql.Driver") \
    .load().limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date_Casted
0,YR7220,NewYork,Electronics,12254943,31.0,20.770000,1.86,29.139999,2019-11-26
1,YR7220,NewYork,Furniture,72619323C,15.0,9.750000,1.50,13.500000,2019-11-26
2,YR7220,NewYork,Electronics,34161682B,88.0,62.480000,4.40,83.599998,2019-11-26
3,YR7220,NewYork,Kitchen,79411621,91.0,58.240002,3.64,87.360001,2019-11-26
4,YR7220,NewYork,Fashion,39520263T,85.0,51.000000,2.55,82.449997,2019-11-26


### Adım4:Temiz veriyi hdfs/user/train/spark_odev_transaction dizinine parquet formatta snappy sıkıştırma ile yazınız.

In [22]:
start_time = time.time()

df2.write.format("parquet") \
.mode("overwrite") \
.option("compression", "snappy") \
.save("hdfs://localhost:9000/user/train/spark_odev_transaction")

print("%s secs = "% (time.time() - start_time))

2.36483097076416 secs = 


In [23]:
! hdfs dfs -ls /user/train/spark_odev_transaction

Found 2 items
-rw-r--r--   1 train supergroup          0 2025-06-14 20:53 /user/train/spark_odev_transaction/_SUCCESS
-rw-r--r--   1 train supergroup     244350 2025-06-14 20:53 /user/train/spark_odev_transaction/part-00000-5221d60c-7c2c-4bfa-8774-9b990acb3e86-c000.snappy.parquet


In [24]:
spark.version

'3.0.0'

In [26]:
spark.stop()